# Financial Corpus

### Download Corpus & Fetch Articles 

Corpus contains articles across the following categories:

* Policy (licenses , regulation, SEC, monetary, fed, monetary,fiscal,imf)

* International  Finance( global finance, IMF, ECB, trouble in Greece, RMB devaluation)

* Economy (GDP, Jobs, unemployment, housing, economy)

* Raising Capital(ipo, equity)

* Real Estate

* Mergers &  Acquisitions (merger,acquisitions)

* Oil(oil,oil prices,natural gas price)

* Commodities (commodities,gold ,silver)

* Fraud(insider trading, ponzi scheme, finance fraud)

* Litigation (company litigation, company settlement,)

* Earning Reports

In [25]:
import requests 
import sqlite3
import pandas as pd
import urllib

url = "https://github.com/skillachie/binaryNLP/raw/master/corpora/finance_corpus.db"
urllib.urlretrieve (url, "finance_corpus.db")

('finance_corpus.db', <httplib.HTTPMessage instance at 0x7fe28b2f27a0>)

In [68]:
# Fetch articles from DB

conn = sqlite3.connect("finance_corpus.db")
df = pd.read_sql("SELECT category, body from articles",conn)

print df.head

categories = []
articles = []

for index, row in df.iterrows():
    
    categories.append(row['category'])
    #(s.decode('unicode_escape').encode('ascii','ignore'))
    articles.append(row['body'])
    

<bound method DataFrame.head of                    category                                               body
0                    Policy  WASHINGTON—The Federal Reserve Board of Govern...
1                    Policy  There should be no rise until the Fed has retu...
2                    Policy  BEFORE the financial crisis life was simple fo...
3                    Policy  James Rickards is a hedge fund manager in New ...
4                    Policy  • The Federal Reserve announced it would launc...
5                    Policy  At its annual meeting of the world's central b...
6                    Policy  Is the Federal Reserve printing money to finan...
7                    Policy  Despite the Federal Reserve’s trillions of dol...
8                    Policy  For the first time in a while, Federal Reserve...
9                    Policy  Just in the nick of time, as 2014 winds down, ...
10                   Policy  FORTUNE — Political dysfunction in D.C. has le...
11                  

# Preprocessing 

Note: Ensure to download stopwords and punkt from NLTK using nltk.download() more at http://www.nltk.org/data.html

In [123]:
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer


def preprocessing(text,stem=False):
    
    
    # Remove punctuations
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
    
    tokens = word_tokenize(text)
    #stop = stopwords.words('english')
    #tokens =[word for word in tokens if word not in stop]
    #tokens = [word.lower() for word in tokens]

    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
        
    return tokens
    

In [131]:
# Preprocess all articles. To be used for Doc2Vec
# categories, articles

#articles_tokens = []
#for article in articles:
#    articles_tokens.append(preprocessing(article))
    
    
        

# Train & Test Model

In [134]:
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import KFold
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
import numpy as np


tfidf_vec = TfidfVectorizer(tokenizer=preprocessing,
                            stop_words=stopwords.words('english'),
                            sublinear_tf=True,
                            use_idf=True)

tokens_vec = tfidf_vec.fit_transform(articles)



X = tokens_vec
le = LabelEncoder()
y = le.fit_transform(categories)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=13)
clf = LinearSVC(C=1.0,loss='squared_hinge',max_iter=2000)

def train():

    cross_val = KFold(len(y),n_folds=3,shuffle=True)
    for train_index, test_index in cross_val:
        X_train,X_test = X[train_index], X[test_index]
        y_train,y_test = y[train_index], y[test_index]

        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        #print classification_report(le.inverse_transform(y_test),le.inverse_transform(y_pred))



def best_parameters():
    
    penalties = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,2.0])
    loss = ['hinge','squared_hinge']
    
    model = LinearSVC()
    grid = GridSearchCV(estimator=model ,n_jobs=5,param_grid=dict(C=penalties,loss=loss,max_iter=[2000]))
    grid.fit(X_train, y_train)
    print(grid)
        
    # summarize the results of the grid search
    print(grid.best_score_)
    print(grid.best_estimator_.C)
    print(grid.best_estimator_.loss)
    
#best_parameters()
train()
  
y_pred = clf.predict(X_test)
print classification_report(le.inverse_transform(y_test),le.inverse_transform(y_pred))


             precision    recall  f1-score   support

    Capital       0.97      0.93      0.95        91
Commodities       0.88      0.90      0.89        78
Earning_Reports       0.94      0.96      0.95        84
    Economy       0.90      0.83      0.87        90
      Fraud       0.92      0.93      0.93        89
International_Finance       0.85      0.89      0.87        84
 Litigation       0.93      0.95      0.94        66
Mergers_Acquisitions       0.98      0.94      0.96        67
        Oil       0.93      0.97      0.95        79
     Policy       0.86      0.85      0.85        86
Real_Estate       0.93      0.89      0.91        28

avg / total       0.92      0.91      0.91       842

